Laboratorio 2 - Entorno estocástico -  Ramiro Sanes (368397) y Joaquín Guerra (307854)

# Evaluación en un entorno estocástico


In [248]:
import gymnasium as gym
from gymnasium.wrappers import RecordEpisodeStatistics, RecordVideo
import numpy as np
import GridWorldEnv

from Utils import print_state_values, print_state_action_values, print_policy_grids

In [249]:
# Algunas constantes

HUMAN_RENDER = False # False si queremos guardar el episodio en un video, True si queremos verlo en tiempo real (no disponible en Google Colab u otros entornos sin interfaz gráfica)

RECORD_VIDEO = False # True si queremos guardar el episodio en un video, quizas se necesita: https://anaconda.org/conda-forge/gymnasium-other
RECORD_EVERY = 1 # Cada cuántos episodios guardamos un video

GRID_SIZE = 4 # Tamaño del grid
GAMMA = 1 # Factor de descuento

## Gymnasium Wrapper

En Gymnasium, los [wrappers](https://gymnasium.farama.org/api/wrappers/) son herramientas que permiten modificar el comportamiento de los entornos sin alterar su código base. Estos *wrappers* se utilizan para extender funcionalidades, como la normalización de observaciones, limitación de acciones o registro de estadísticas y videos durante el entrenamiento de agentes.

Para este ejercicio, se utilizarán dos *wrappers* de Gymnasium:

**`RecordVideo`**: Este *wrapper* permite grabar videos de las ejecuciones del agente en el entorno. Es especialmente útil para visualizar el comportamiento del agente y evaluar su desempeño. Se puede configurar para grabar todos los episodios o solo episodios específicos, según una función de activación definida por el usuario.

Documentación oficial de Gymnasium sobre [grabación de agentes](https://gymnasium.farama.org/introduction/record_agent/) y la [API de wrappers](https://gymnasium.farama.org/api/wrappers/). 

In [250]:
def get_environment(human_render=HUMAN_RENDER, record_video=RECORD_VIDEO, record_every=RECORD_EVERY, grid_size=GRID_SIZE):
    assert not (human_render and record_video), "No se puede renderizar en tiempo real y guardar un video a la vez"
    
    render_mode = "human" if human_render else "rgb_array"
    
    # Initialise the environment
    env = gym.make("gymnasium_env/GridWorld_stochastic-v0", render_mode=render_mode, size=grid_size)

    if record_video:
        env = RecordVideo(env, video_folder="./videos", name_prefix="gridworld",
                    episode_trigger=lambda x: x % record_every == 0)
    
    return env

Probamos el entorno y sus wrappers con un agente aleatorio para visualizar el comportamiento del entorno y la grabación de videos.

In [251]:
# export IMAGEIO_FFMPEG_EXE
#import os
#os.environ["IMAGEIO_FFMPEG_EXE"] = "/opt/homebrew/bin/ffmpeg"

In [252]:
env = get_environment()

# We play 10 episodes, each one with a random policy
for episode_num in range(10):
    obs, info = env.reset()

    episode_over = False
    while not episode_over:
        action = env.action_space.sample()  # random policy
        obs, reward, terminated, truncated, info = env.step(action)
        episode_over = terminated or truncated

env.close()

## Representación de la Política $\pi$

Para implementar los algoritmos de programación dinámica, necesitamos definir cómo representaremos la política $\pi(s, a)$. 

### Estructura de la Política
La política se almacenará en un **diccionario** de la forma:

```python
pi = {
    ((x, y), a): probabilidad
}
```

Donde:
- $(x, y)$ representa el estado en la grilla.
- $a$ representa una de las cuatro acciones posibles.
- `probabilidad` es la probabilidad de tomar la acción $a$ en el estado $(x, y)$.

Función auxiliar para saber si estamos en un estado final

In [253]:
def is_done(state):
    return state in [(0, 0), (GRID_SIZE - 1, GRID_SIZE - 1)]

### Ejemplo: Política Uniforme
Una política uniforme en la que todas las acciones tienen la misma probabilidad ($ 25\% $) se define como:

In [254]:
pi_rand = {}
for x in range(GRID_SIZE):
    for y in range(GRID_SIZE):
        for a in range(4):  # Cuatro acciones posibles
            if is_done((x, y)): # Si no estamos en un estado terminal
                continue
            pi_rand[((x, y), a)] = 1 / 4  # Probabilidad uniforme

print(pi_rand)

{((0, 1), 0): 0.25, ((0, 1), 1): 0.25, ((0, 1), 2): 0.25, ((0, 1), 3): 0.25, ((0, 2), 0): 0.25, ((0, 2), 1): 0.25, ((0, 2), 2): 0.25, ((0, 2), 3): 0.25, ((0, 3), 0): 0.25, ((0, 3), 1): 0.25, ((0, 3), 2): 0.25, ((0, 3), 3): 0.25, ((1, 0), 0): 0.25, ((1, 0), 1): 0.25, ((1, 0), 2): 0.25, ((1, 0), 3): 0.25, ((1, 1), 0): 0.25, ((1, 1), 1): 0.25, ((1, 1), 2): 0.25, ((1, 1), 3): 0.25, ((1, 2), 0): 0.25, ((1, 2), 1): 0.25, ((1, 2), 2): 0.25, ((1, 2), 3): 0.25, ((1, 3), 0): 0.25, ((1, 3), 1): 0.25, ((1, 3), 2): 0.25, ((1, 3), 3): 0.25, ((2, 0), 0): 0.25, ((2, 0), 1): 0.25, ((2, 0), 2): 0.25, ((2, 0), 3): 0.25, ((2, 1), 0): 0.25, ((2, 1), 1): 0.25, ((2, 1), 2): 0.25, ((2, 1), 3): 0.25, ((2, 2), 0): 0.25, ((2, 2), 1): 0.25, ((2, 2), 2): 0.25, ((2, 2), 3): 0.25, ((2, 3), 0): 0.25, ((2, 3), 1): 0.25, ((2, 3), 2): 0.25, ((2, 3), 3): 0.25, ((3, 0), 0): 0.25, ((3, 0), 1): 0.25, ((3, 0), 2): 0.25, ((3, 0), 3): 0.25, ((3, 1), 0): 0.25, ((3, 1), 1): 0.25, ((3, 1), 2): 0.25, ((3, 1), 3): 0.25, ((3, 2), 0)

### Ejemplo: Política Determinista (derecha y abajo)
Si queremos que el agente siempre elija moverse a la derecha (acción `0`) a excepción de la ultima columna que se mueve hacia abajo (accion `3`), podemos definir la política de la siguiente manera:

In [255]:
pi_der_aba = {}
for x in range(GRID_SIZE):
    for y in range(GRID_SIZE):
        if is_done((x, y)): # Si no estamos en un estado terminal
            continue
        if x == GRID_SIZE - 1:  # Si estamos en la última columna, solo podemos movernos hacia abajo
            pi_der_aba[((x, y), 3)] = 1
        else:  # En otro caso, podemos movernos hacia la derecha
            pi_der_aba[((x, y), 0)] = 1.0
print(pi_der_aba)

{((0, 1), 0): 1.0, ((0, 2), 0): 1.0, ((0, 3), 0): 1.0, ((1, 0), 0): 1.0, ((1, 1), 0): 1.0, ((1, 2), 0): 1.0, ((1, 3), 0): 1.0, ((2, 0), 0): 1.0, ((2, 1), 0): 1.0, ((2, 2), 0): 1.0, ((2, 3), 0): 1.0, ((3, 0), 3): 1, ((3, 1), 3): 1, ((3, 2), 3): 1}


### Ejemplo: Política Determinista (siempre abajo)
Si queremos que el agente siempre elija moverse hacia abajo (acción `3`), podemos definir la política de la siguiente manera:

In [256]:
pi_abajo = {}
for x in range(GRID_SIZE):
    for y in range(GRID_SIZE):
        if is_done((x, y)): # Si no estamos en un estado terminal
                continue
        pi_abajo[((x, y), 3)] = 1

## Elección de una acción dado un estado y una política
Para elegir una acción dado un estado y una política, necesitamos muestrear una acción de acuerdo a la distribución de probabilidad definida en la política. Para esto, podemos utilizar la función `np.random.choice` de NumPy, que nos permite muestrear un elemento de un conjunto dado con una probabilidad dada.

Más información sobre `np.random.choice` en la [documentación oficial de NumPy](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html).

In [257]:
def select_action(state, policy):
    """
    Dado un estado y una política, devuelve la acción a tomar.

    Parámetros:
    - state: Tupla (x, y) representando el estado actual.
    - policy: Diccionario {((x, y), a): probabilidad} con la política.

    Retorna:
    - La acción seleccionada según la política.
    
    Nota: Si la política es estocástica, se selecciona una acción al azar según las probabilidades.
    """
    actions = range(4)

    prob = [policy.get((state, a), 0) for a in actions]

    return np.random.choice(actions, p=prob)

select_action((2,2), pi_rand)

np.int64(2)

## Dinámica del Ambiente ($ p $) 

**La dinámica del ambiente $ p(s' | s, a) $**, que define las transiciones entre estados al tomar una acción. Esta información nos permite conocer, para cada estado y acción, cuál será la probabilidad de llegar al próximo estado y la recompensa recibida.

> Cuando creamos un ambiente con `gym.make()`, Gymnasium aplica un envoltorio (`OrderEnforcing`) que restringe el acceso directo a ciertos atributos internos. Para acceder a `p`, necesitamos usar `env.unwrapped`, que nos da acceso al objeto subyacente sin restricciones.

Ejemplo de acceso a la política desde el ambiente:


In [258]:
p = env.unwrapped.p
print(p)

{((0, 0), 0): [(1.0, (0, 0), 0.0)], ((0, 0), 1): [(1.0, (0, 0), 0.0)], ((0, 0), 2): [(1.0, (0, 0), 0.0)], ((0, 0), 3): [(1.0, (0, 0), 0.0)], ((0, 1), 0): [(0.8, (1, 1), -1.0), (0.1, (0, 0), -1.0), (0.1, (0, 2), -1.0)], ((0, 1), 1): [(0.8, (0, 0), -1.0), (0.1, (0, 0), -1.0), (0.1, (0, 2), -1.0)], ((0, 1), 2): [(0.8, (0, 1), -1.0), (0.1, (0, 0), -1.0), (0.1, (0, 2), -1.0)], ((0, 1), 3): [(0.8, (0, 2), -1.0), (0.1, (0, 0), -1.0), (0.1, (0, 2), -1.0)], ((0, 2), 0): [(0.8, (1, 2), -1.0), (0.1, (0, 1), -1.0), (0.1, (0, 3), -1.0)], ((0, 2), 1): [(0.8, (0, 1), -1.0), (0.1, (0, 1), -1.0), (0.1, (0, 3), -1.0)], ((0, 2), 2): [(0.8, (0, 2), -1.0), (0.1, (0, 1), -1.0), (0.1, (0, 3), -1.0)], ((0, 2), 3): [(0.8, (0, 3), -1.0), (0.1, (0, 1), -1.0), (0.1, (0, 3), -1.0)], ((0, 3), 0): [(0.8, (1, 3), -1.0), (0.1, (0, 2), -1.0), (0.1, (0, 3), -1.0)], ((0, 3), 1): [(0.8, (0, 2), -1.0), (0.1, (0, 2), -1.0), (0.1, (0, 3), -1.0)], ((0, 3), 2): [(0.8, (0, 3), -1.0), (0.1, (0, 2), -1.0), (0.1, (0, 3), -1.0)], (

El diccionario tiene la siguiente forma:

```python
self.p = {
    ((x, y), a): [(probabilidad, (nuevo_x, nuevo_y), recompensa)]
}
```

Donde:
- `((x, y), a)`: Clave que representa el **estado actual** `(x, y)` y la **acción** `a` tomada.
- **Valor asociado**: Una lista con **una o más transiciones posibles** en el formato:
  - `probabilidad`: La probabilidad de que ocurra la transición (en este caso, siempre `1.0`, porque el ambiente es determinista).
  - `(nuevo_x, nuevo_y)`: El nuevo estado después de tomar la acción.
  - `recompensa`: El valor de recompensa por realizar la acción.

En este caso al tener una dinámica del ambiente estocástica, observamos que en algunos estados, tomando alguna acción se puede llegar a 3 distintos pares (estado siguiente, recompensa)

In [259]:
max(len(i) for i in p.values())

3

## Policy Iteration

La **Iteración de Política** (*Policy Iteration*) es un método fundamental en **Programación Dinámica** para encontrar la política óptima $\pi^*$ en un proceso de decisión de Markov (*MDP*) finito. Se basa en dos pasos clave que se repiten iterativamente hasta la convergencia:

1. **Evaluación de Política** (*Policy Evaluation*): Se calcula el valor de la política actual $\pi$, es decir, se obtiene la función de valor $V_\pi(s)$ resolviendo la ecuación de Bellman para todos los estados:
   $$
   V_\pi(s) = \sum_{a} \pi(a | s) \sum_{s', r} p(s', r | s, a) \left[ r + \gamma V_\pi(s') \right]
   $$
   donde $p(s', r | s, a)$ representa la dinámica del ambiente y $\gamma$ es el factor de descuento.

2. **Mejora de Política** (*Policy Improvement*): Se actualiza la política eligiendo en cada estado la acción que maximiza el valor esperado según la función de acción-valor $Q_\pi(s, a)$:
   $$
   \pi'(s) = \arg\max_{a} \sum_{s', r} p(s', r | s, a) \left[ r + \gamma V_\pi(s') \right]
   $$
   Si la nueva política $\pi'$ es diferente de la anterior, se repite el proceso con la nueva política. Si no cambia, hemos encontrado la política óptima $\pi^*$.

Proceso Iterativo:
1. Inicializar una política arbitraria $\pi$.
2. Aplicar **Policy Evaluation** para calcular $V_\pi$.
3. Aplicar **Policy Improvement** para obtener una nueva política $\pi'$.
4. Si $\pi' = \pi$, detenerse. De lo contrario, repetir desde el paso 2.

Este algoritmo garantiza la convergencia a la política óptima en un número finito de iteraciones en ambientes con estados y acciones finitas.

### Implementación de Policy evaluation

$$
\begin{aligned}
\textbf{Input:} \quad & \pi,\ \text{la política a evaluar.} \\
\textbf{Parámetro:} \quad & \theta > 0,\ \text{umbral pequeño que determina la precisión.} \\
\textbf{Inicializar:} \quad & V(s) \text{ arbitrariamente para todo } s \in S^+, \text{ excepto } V(\text{terminal}) = 0. \\
\\
\textbf{Repetir:} \quad & \Delta \gets 0 \\
& \text{Para cada } s \in S: \\
& \quad\quad v \gets V(s) \\
& \quad\quad V(s) \gets \sum_{a} \pi(a \mid s) \sum_{s',r} p(s',r \mid s,a)\,\Bigl[r + \gamma V(s')\Bigr] \\
& \quad\quad \Delta \gets \max\Bigl(\Delta,\, \bigl|v - V(s)\bigr|\Bigr) \\
\\
\textbf{Hasta que:} \quad & \Delta < \theta
\end{aligned}
$$



In [260]:
def policy_evaluation(p, pi, gamma=GAMMA, theta=1e-5, size=GRID_SIZE):
    """
    Evalúa la política 'pi' en el entorno 'env' usando el método de evaluación iterativa.
    
    Parámetros:
        p: dinámica del entorno, diccionario que mapea (estado, acción) a una lista de (probabilidad, estado, recompensa).
        pi: politica a evaluar, diccionario que mapea (estado, acción) a su probabilidad en la política.
        gamma: factor de descuento.
        theta: umbral de convergencia.
    
    Retorna:
        V: función de valor de la política 'pi', diccionario que mapea estado (x, y) a su valor.
    """
    V = {}
    q = {}


    for y in range(size):
        for x in range(size):
            s = (x,y)
            V[s] = 0

    while True:
        delta = 0
        for y in range(size):
            for x in range(size):
                s = (x,y)
                v = V[s]
                pol = [pi.get((s, a), 0) for a in range(4)]

                for a in range(4):
                    q[(s,a)] = sum([prob * (r + gamma * V[s_siguiente]) for prob, s_siguiente, r in p[(s, a)]])

                V[s] = sum([pol[a] * q[(s,a)] for a in range(4)])
                delta = max(delta, abs(v - V[s]))

        if delta < theta:
            break

    return V

    
            


In [261]:
[pi_abajo.get(((0,1), a), 0) for a in range(4)]

#print(pi_abajo[(0,1),1])

[0, 0, 0, 1]

In [262]:
V_da_pi = policy_evaluation(p, pi_der_aba)
print_state_values(V_da_pi, GRID_SIZE)

    0.00    -5.85    -4.72    -3.59
   -5.44    -4.93    -3.71    -2.48
   -4.98    -3.78    -2.51    -1.25
   -4.12    -2.77    -1.39     0.00


In [263]:
V_d_pi = policy_evaluation(p, pi_abajo, gamma=0.99)
print_state_values(V_d_pi, GRID_SIZE)

    0.00  -100.00  -100.00    -3.54
  -81.58  -100.00  -100.00    -2.46
  -90.44  -100.00  -100.00    -1.24
  -91.32  -100.00  -100.00     0.00


In [264]:
V_pi_random = policy_evaluation(p, pi_rand, gamma=0.99)
print_state_values(V_pi_random, GRID_SIZE)

    0.00   -12.97   -17.59   -18.61
  -11.23   -15.60   -17.22   -16.55
  -16.55   -17.22   -15.60   -11.23
  -18.61   -17.59   -12.97     0.00


### Implementación de Policy Improvement

$$
\begin{aligned}
\textbf{Input:} \quad & V,\ \text{la función de valor evaluada}, \\
& p,\ \text{la dinamica del ambiente } \\
& \gamma,\ \text{el factor de descuento.} \\
\\
\textbf{Inicializar:} \quad & \text{Para cada } s \in S \text{ y } a \in A(s),\ new\_pi(s,a) \text{ se asigna arbitrariamente.} \\
\\
\textbf{Para cada } s \in S \text{ (excepto estados terminales):} \quad & \\
& \quad \text{Para cada acción } a \in A(s): \\
& \quad\quad Q(s,a) \gets \sum_{s',r} p(s',r \mid s,a) \Bigl[ r + \gamma\, V(s') \Bigr] \\
& \quad \text{Definir } best\_actions = \{ a \in A(s) \mid Q(s,a) = \max_{a' \in A(s)} Q(s,a') \} \\
& \quad \text{Para cada } a \in A(s): \\
& \quad\quad new\_pi(s,a) \gets 
\begin{cases}
\displaystyle \frac{1}{\lvert best\_actions \rvert}, & \text{si } a \in best\_actions, \\
0, & \text{si } a \notin best\_actions.
\end{cases} \\
\\
\textbf{Retornar:} \quad & new\_pi.
\end{aligned}
$$


In [265]:
def calculate_Q(p, V, gamma=GAMMA, size=GRID_SIZE):
    """
    Calcula el valor Q(s, a) para cada estado y acción.
    
    Parámetros:
        p: dinámica del entorno, diccionario que mapea (estado, acción) a una lista de (probabilidad, estado, recompensa).
        V: función de valor, diccionario que mapea estado (x, y) a su valor.
        gamma: factor de descuento.
    
    Retorna:
        Q: valor Q(s, a).
    """
    Q = {}

    for y in range(size):
        for x in range(size):
            s = (x, y)
            for a in range(4):
                Q[(s, a)] = sum([prob * (r + gamma * V[next_s]) for (prob,next_s,r) in p[(s, a)]])

    return Q

In [266]:
Q_pi_random = calculate_Q(p, V_pi_random)
print_state_action_values(Q_pi_random, GRID_SIZE)

Valores para la acción Derecha (acción 0):
    0.00   -17.93   -19.37   -19.41
  -15.14   -17.80   -17.56   -17.22
  -17.76   -16.80   -13.00   -11.64
  -18.59   -14.86    -3.86     0.00


Valores para la acción Arriba (acción 1):
    0.00   -14.23   -18.56   -19.41
   -2.65   -14.39   -18.39   -18.87
  -12.97   -16.80   -17.80   -15.89
  -17.76   -18.26   -16.34     0.00


Valores para la acción Izquierda (acción 2):
    0.00    -3.86   -14.86   -18.59
  -11.64   -13.00   -16.80   -17.76
  -17.22   -17.56   -17.80   -15.14
  -19.41   -19.37   -17.93     0.00


Valores para la acción Abajo (acción 3):
    0.00   -16.34   -18.26   -17.76
  -15.89   -17.80   -16.80   -12.97
  -18.87   -18.39   -14.39    -2.65
  -19.41   -18.56   -14.23     0.00




In [267]:
Q_da_pi = calculate_Q(p, V_da_pi)
print_state_action_values(Q_da_pi, GRID_SIZE)

Valores para la acción Derecha (acción 0):
    0.00    -5.85    -4.72    -4.48
   -5.44    -4.93    -3.71    -3.47
   -4.98    -3.78    -2.51    -2.25
   -4.12    -2.77    -1.39     0.00


Valores para la acción Arriba (acción 1):
    0.00    -6.76    -5.62    -4.48
   -1.50    -6.64    -5.50    -4.36
   -6.31    -5.71    -4.48    -3.23
   -5.89    -4.68    -3.40     0.00


Valores para la acción Izquierda (acción 2):
    0.00    -2.08    -6.52    -5.38
   -5.85    -6.32    -5.67    -4.45
   -5.94    -5.75    -4.53    -3.26
   -5.21    -4.95    -3.60     0.00


Valores para la acción Abajo (acción 3):
    0.00    -6.02    -4.81    -3.59
   -5.48    -4.98    -3.73    -2.48
   -5.25    -3.98    -2.62    -1.25
   -5.21    -3.87    -2.50     0.00




In [268]:
Q_d_pi = calculate_Q(p, V_d_pi, gamma=0.99)
print_state_action_values(Q_d_pi, GRID_SIZE)

Valores para la acción Derecha (acción 0):
    0.00  -100.00   -23.60    -4.40
  -89.15  -100.00   -22.75    -3.42
  -97.32  -100.00   -21.78    -2.23
  -98.19  -100.00   -20.80     0.00


Valores para la acción Arriba (acción 1):
    0.00  -100.00  -100.00    -4.40
   -9.95  -100.00  -100.00    -4.28
  -82.73  -100.00  -100.00    -3.19
  -90.62  -100.00  -100.00     0.00


Valores para la acción Izquierda (acción 2):
    0.00   -20.80  -100.00   -80.79
  -74.57   -85.41  -100.00   -80.67
  -89.75   -92.43  -100.00   -80.44
  -91.32   -93.12  -100.00     0.00


Valores para la acción Abajo (acción 3):
    0.00  -100.00  -100.00    -3.54
  -81.58  -100.00  -100.00    -2.46
  -90.44  -100.00  -100.00    -1.24
  -91.32  -100.00  -100.00     0.00




### Implementación de Policy Improvement

$$
\begin{aligned}
\textbf{Input:} \quad & V,\ \text{la función de valor evaluada}, \\
& p,\ \text{la dinamica del ambiente } \\
& \gamma,\ \text{el factor de descuento.} \\
\\
\textbf{Inicializar:} \quad & \text{Para cada } s \in S \text{ y } a \in A(s),\ new\_pi(s,a) \text{ se asigna arbitrariamente.} \\
\\
\textbf{Para cada } s \in S \text{ (excepto estados terminales):} \quad & \\
& \quad \text{Para cada acción } a \in A(s): \\
& \quad\quad Q(s,a) \gets \sum_{s',r} p(s',r \mid s,a) \Bigl[ r + \gamma\, V(s') \Bigr] \\
& \quad \text{Definir } best\_actions = \{ a \in A(s) \mid Q(s,a) = \max_{a' \in A(s)} Q(s,a') \} \\
& \quad \text{Para cada } a \in A(s): \\
& \quad\quad new\_pi(s,a) \gets 
\begin{cases}
\displaystyle \frac{1}{\lvert best\_actions \rvert}, & \text{si } a \in best\_actions, \\
0, & \text{si } a \notin best\_actions.
\end{cases} \\
\\
\textbf{Retornar:} \quad & new\_pi.
\end{aligned}
$$


In [269]:
def improve_policy(p, V, gamma=GAMMA, size=GRID_SIZE):
    """
    Genera una política mejorada (greedy) a partir de la función de valor V.
    
    Parámetros:
        env: entorno que posee un atributo 'p' con la dinámica.
        V: diccionario que mapea estados a sus valores.
        gamma: factor de descuento.
    
    Retorna:
        new_pi: diccionario que representa la política mejorada, mapeando (estado, acción) a probabilidad.
    """

    new_pi = {}
    Q = calculate_Q(p, V, gamma)

    for y in range(size):
        for x in range(size):
            s = (x, y)
            if is_done(s):
                continue
            lista = np.array([Q[(s, a)] for a in range(4)])
            lista_indices = np.where(lista == np.max(lista))[0]
            for a in range(4):
                #new_pi[(s, a)] = 1 if a == mejor_a else 0
                new_pi[(s, a)] = 1/len(lista_indices) if a in lista_indices else 0
    
    return new_pi


In [270]:
def compare_policies(pi_1, pi_2, tol=1e-6, size=GRID_SIZE):
    """
    Compara dos políticas y retorna True si son iguales (considerando una tolerancia en los valores), o False en caso contrario.
    
    Parámetros:
        pi_1, pi_2: diccionarios que representan las políticas (mapean (estado, acción) a probabilidad).
        tol: tolerancia para comparar las probabilidades.
    
    Retorna:
        True si ambas políticas son iguales en todos los (estado, acción); False de lo contrario.
    """
    delta = 0
    for y in range(size):
        for x in range(size):
            s = (x, y)
            for a in range(4):
                if is_done(s):
                    continue
                # Comparamos las probabilidades de ambas políticas para el estado s y la acción a
                delta = max(delta, abs(pi_1.get((s, a), 0) - pi_2.get((s, a), 0)))

    

    return delta < tol

In [271]:
def policy_iteration(p, pi_init, gamma=GAMMA, theta=1e-5, size=GRID_SIZE):
    """
    Realiza el algoritmo de iteración de política.
    
    Parámetros:
        p: dinámica del entorno, diccionario que mapea (estado, acción) a una lista de (probabilidad, estado, recompensa).
        gamma: factor de descuento.
        theta: umbral de convergencia.
    
    Retorna:
        pi: política óptima.
    """
    pi = pi_init
    iteration_counter = 0

    while True:
        V = policy_evaluation(p, pi, gamma, theta, size)
        new_pi = improve_policy(p, V, gamma=GAMMA, size=GRID_SIZE)
        if compare_policies(pi, new_pi):
            break
        else:
            pi = new_pi
            iteration_counter += 1
        
        

    return pi, iteration_counter

In [272]:
pi_star = policy_iteration(p, pi_rand)[0]
print_policy_grids(pi_star, GRID_SIZE)

Política para la acción Derecha (acción 0):
  0.00   0.00   0.00   0.00
  0.00   0.00   1.00   0.00
  0.00   0.00   1.00   0.00
  0.00   1.00   1.00   0.00


Política para la acción Arriba (acción 1):
  0.00   0.00   0.00   0.00
  1.00   0.00   0.00   0.00
  1.00   0.00   0.00   0.00
  1.00   0.00   0.00   0.00


Política para la acción Izquierda (acción 2):
  0.00   1.00   1.00   0.00
  0.00   1.00   0.00   0.00
  0.00   1.00   0.00   0.00
  0.00   0.00   0.00   0.00


Política para la acción Abajo (acción 3):
  0.00   0.00   0.00   1.00
  0.00   0.00   0.00   1.00
  0.00   0.00   0.00   1.00
  0.00   0.00   0.00   0.00




In [273]:
env = get_environment()

for episode_num in range(10):
    obs, info = env.reset()

    episode_over = False
    while not episode_over:
        state = (obs["pos"][0], obs["pos"][1])
        action = select_action(state, pi_star)
        obs, reward, terminated, truncated, info = env.step(action)
        episode_over = terminated or truncated

env.close()

## Value Iteration

Value Iteration es un algoritmo de programación dinámica empleado para estimar la función de valor óptima $ V^* $ y, a partir de ella, derivar la política óptima $ \pi^* $. Este método se basa en actualizar iterativamente los valores de cada estado utilizando la ecuación de Bellman, hasta que las actualizaciones sean menores que un pequeño umbral $ \theta $ que determina la precisión de la estimación.

El proceso general es el siguiente:

1. **Inicialización:**  
   Se asigna un valor arbitrario a $ V(s) $ para todos los estados $ s \in S^+ $, excepto en los estados terminales, donde se establece $ V(\text{terminal}) = 0 $.

2. **Actualización Iterativa:**  
   Para cada estado $ s \in S $, se actualiza su valor mediante la fórmula:
   $$
   V(s) \leftarrow \max_{a} \sum_{s', r} p(s', r \mid s, a) \Bigl[ r + \gamma V(s') \Bigr],
   $$
   donde $ p(s', r \mid s, a) $ es la probabilidad de transitar al estado $ s' $ y recibir la recompensa $ r $ al tomar la acción $ a $ en el estado $ s $.  
   La iteración continúa hasta que la diferencia máxima entre los valores antiguos y actualizados en todos los estados sea menor que $ \theta $.

3. **Extracción de la Política:**  
   Una vez convergido $ V $, se define la política óptima determinista $ \pi^* $ de la siguiente manera:
   $$
   \pi^*(s) = \operatorname*{argmax}_{a} \sum_{s', r} p(s', r \mid s, a) \Bigl[ r + \gamma V(s') \Bigr].
   $$

Este algoritmo es fundamental en el aprendizaje por refuerzo y en la resolución de procesos de decisión de Markov, ya que permite obtener de manera eficiente tanto la función de valor óptima como la política que maximiza el retorno esperado en cada estado.


In [274]:
def value_iteration(p, gamma=GAMMA, theta=1e-5, size=GRID_SIZE):
    """
    Realiza el algoritmo de iteración de valor.
    
    Parámetros:
        p: dinámica del entorno, diccionario que mapea (estado, acción) a una lista de (probabilidad, estado, recompensa).
        gamma: factor de descuento.
        theta: umbral de convergencia.
    
    Retorna:
        pi: la pólitica óptima.
    """


    # Inicializamos la función de valor V y la política pi
    V = {(x,y):0 for x in range(size) for y in range(size)}
    V[(0,0)] = 0
    V[(size-1,size-1)] = 0
    V_prev = V.copy()
    dict_actions = {(x,y):[] for x in range(size) for y in range(size)}
    pi = {}


    while True:
        #print("entro")

        q = calculate_Q(p, V_prev, gamma, size)
        
        for x in range(size):
            for y in range(size):
                s = (x, y)
                if is_done(s):
                    continue
                
                #print(calculate_Q(p, V_prev, gamma, size))

                
                q_array = np.array([q[s,a] for a in range(4)])
                max_indexes = np.where(q_array == np.max(q_array))[0]

                #if np.max(q_array) >= V[s]:

                dict_actions[s] = max_indexes.tolist()
                V[s] = np.max(q_array)
        

        # Comprobamos la convergencia

        delta = max(abs(V[s] - V_prev[s]) for s in V.keys())
        if delta < theta:
            break
        else:
            V_prev = V.copy()   

    for state,actions in dict_actions.items():
        for action in actions:
                pi[(state, action)] = 1 / len(actions)

    return pi
    


            
            

In [275]:
pi_star2 = value_iteration(p)
print_policy_grids(pi_star2, GRID_SIZE)

Política para la acción Derecha (acción 0):
  0.00   0.00   0.00   0.00
  0.00   0.00   1.00   0.00
  0.00   0.00   1.00   0.00
  0.00   1.00   1.00   0.00


Política para la acción Arriba (acción 1):
  0.00   0.00   0.00   0.00
  1.00   0.00   0.00   0.00
  1.00   0.00   0.00   0.00
  1.00   0.00   0.00   0.00


Política para la acción Izquierda (acción 2):
  0.00   1.00   1.00   0.00
  0.00   1.00   0.00   0.00
  0.00   1.00   0.00   0.00
  0.00   0.00   0.00   0.00


Política para la acción Abajo (acción 3):
  0.00   0.00   0.00   1.00
  0.00   0.00   0.00   1.00
  0.00   0.00   0.00   1.00
  0.00   0.00   0.00   0.00




In [276]:
env = get_environment()

for episode_num in range(10):
    obs, info = env.reset()

    episode_over = False
    while not episode_over:
        state = (obs["pos"][0], obs["pos"][1])
        action = select_action(state, pi_star2)
        obs, reward, terminated, truncated, info = env.step(action)
        episode_over = terminated or truncated

env.close()

Imprimimos la dinámica del ambiente estocástico 

In [277]:
p

{((0, 0), 0): [(1.0, (0, 0), 0.0)],
 ((0, 0), 1): [(1.0, (0, 0), 0.0)],
 ((0, 0), 2): [(1.0, (0, 0), 0.0)],
 ((0, 0), 3): [(1.0, (0, 0), 0.0)],
 ((0, 1), 0): [(0.8, (1, 1), -1.0), (0.1, (0, 0), -1.0), (0.1, (0, 2), -1.0)],
 ((0, 1), 1): [(0.8, (0, 0), -1.0), (0.1, (0, 0), -1.0), (0.1, (0, 2), -1.0)],
 ((0, 1), 2): [(0.8, (0, 1), -1.0), (0.1, (0, 0), -1.0), (0.1, (0, 2), -1.0)],
 ((0, 1), 3): [(0.8, (0, 2), -1.0), (0.1, (0, 0), -1.0), (0.1, (0, 2), -1.0)],
 ((0, 2), 0): [(0.8, (1, 2), -1.0), (0.1, (0, 1), -1.0), (0.1, (0, 3), -1.0)],
 ((0, 2), 1): [(0.8, (0, 1), -1.0), (0.1, (0, 1), -1.0), (0.1, (0, 3), -1.0)],
 ((0, 2), 2): [(0.8, (0, 2), -1.0), (0.1, (0, 1), -1.0), (0.1, (0, 3), -1.0)],
 ((0, 2), 3): [(0.8, (0, 3), -1.0), (0.1, (0, 1), -1.0), (0.1, (0, 3), -1.0)],
 ((0, 3), 0): [(0.8, (1, 3), -1.0), (0.1, (0, 2), -1.0), (0.1, (0, 3), -1.0)],
 ((0, 3), 1): [(0.8, (0, 2), -1.0), (0.1, (0, 2), -1.0), (0.1, (0, 3), -1.0)],
 ((0, 3), 2): [(0.8, (0, 3), -1.0), (0.1, (0, 2), -1.0), (0.1, (0,

In [278]:
print_state_values(V_d_pi, GRID_SIZE)
print("\n\n")



pol_abajo,iterations_abajo = policy_iteration(p, pi_abajo, 0.9, theta=1e-5, size=GRID_SIZE)

print(f"Cantidad de Iteraciones: {iterations_abajo} \n")
print_policy_grids(pol_abajo, GRID_SIZE)


    0.00  -100.00  -100.00    -3.54
  -81.58  -100.00  -100.00    -2.46
  -90.44  -100.00  -100.00    -1.24
  -91.32  -100.00  -100.00     0.00



Cantidad de Iteraciones: 2 

Política para la acción Derecha (acción 0):
  0.00   0.00   0.00   0.00
  0.00   0.00   1.00   0.00
  0.00   0.00   1.00   0.00
  0.00   1.00   1.00   0.00


Política para la acción Arriba (acción 1):
  0.00   0.00   0.00   0.00
  1.00   0.00   0.00   0.00
  1.00   0.00   0.00   0.00
  1.00   0.00   0.00   0.00


Política para la acción Izquierda (acción 2):
  0.00   1.00   1.00   0.00
  0.00   1.00   0.00   0.00
  0.00   1.00   0.00   0.00
  0.00   0.00   0.00   0.00


Política para la acción Abajo (acción 3):
  0.00   0.00   0.00   1.00
  0.00   0.00   0.00   1.00
  0.00   0.00   0.00   1.00
  0.00   0.00   0.00   0.00




In [279]:
print_state_values(V_da_pi, GRID_SIZE)
print("\n\n")

pol_der_aba,iterations_der_aba = policy_iteration(p, pi_der_aba, 0.9, theta=1e-5, size=GRID_SIZE)

print(f"Cantidad de Iteraciones: {iterations_der_aba} \n")
print_policy_grids(pol_der_aba, GRID_SIZE)



    0.00    -5.85    -4.72    -3.59
   -5.44    -4.93    -3.71    -2.48
   -4.98    -3.78    -2.51    -1.25
   -4.12    -2.77    -1.39     0.00



Cantidad de Iteraciones: 4 

Política para la acción Derecha (acción 0):
  0.00   0.00   0.00   0.00
  0.00   0.00   1.00   0.00
  0.00   0.00   1.00   0.00
  0.00   1.00   1.00   0.00


Política para la acción Arriba (acción 1):
  0.00   0.00   0.00   0.00
  1.00   0.00   0.00   0.00
  1.00   0.00   0.00   0.00
  1.00   0.00   0.00   0.00


Política para la acción Izquierda (acción 2):
  0.00   1.00   1.00   0.00
  0.00   1.00   0.00   0.00
  0.00   1.00   0.00   0.00
  0.00   0.00   0.00   0.00


Política para la acción Abajo (acción 3):
  0.00   0.00   0.00   1.00
  0.00   0.00   0.00   1.00
  0.00   0.00   0.00   1.00
  0.00   0.00   0.00   0.00




In [280]:
print_state_values(V_pi_random, GRID_SIZE)
print("\n\n")

pol_rand,iterations_rand = policy_iteration(p, pi_rand, 0.9, theta=1e-5, size=GRID_SIZE)

print(f"Cantidad de Iteraciones: {iterations_rand} \n")
print_policy_grids(pol_rand, GRID_SIZE)



    0.00   -12.97   -17.59   -18.61
  -11.23   -15.60   -17.22   -16.55
  -16.55   -17.22   -15.60   -11.23
  -18.61   -17.59   -12.97     0.00



Cantidad de Iteraciones: 2 

Política para la acción Derecha (acción 0):
  0.00   0.00   0.00   0.00
  0.00   0.00   1.00   0.00
  0.00   0.00   1.00   0.00
  0.00   1.00   1.00   0.00


Política para la acción Arriba (acción 1):
  0.00   0.00   0.00   0.00
  1.00   0.00   0.00   0.00
  1.00   0.00   0.00   0.00
  1.00   0.00   0.00   0.00


Política para la acción Izquierda (acción 2):
  0.00   1.00   1.00   0.00
  0.00   1.00   0.00   0.00
  0.00   1.00   0.00   0.00
  0.00   0.00   0.00   0.00


Política para la acción Abajo (acción 3):
  0.00   0.00   0.00   1.00
  0.00   0.00   0.00   1.00
  0.00   0.00   0.00   1.00
  0.00   0.00   0.00   0.00




**3. [extra] Evaluación en un entorno estocástico**
- ¿Cómo cambia la política óptima en este caso?

En este caso podemos observar, en primer lugar, que la política óptima, a diferencia con el caso anterior, es deterministica.
Esto ocurre debido a que por cómo esta definido el ambiente estocastico, al realizar cualquier acción, el agente tiene un 10% de probabilidad de ir hacia arriba, un 10% de ir hacia abajo y un 80% de ir hacia la dirección "deseada". Por lo tanto a diferencia que en el ambiente determinístico, los estados con mejores valores luego de los terminales son el (1,0) y el (2,3) ya que la aleatoriedad les otorga un 10% de probabilidades extra de llegar al estado terminal sobre los otros estados que están pegados al terminal como lo son el (0,1) y el (3,2)